In [11]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=["telepotr_bridging_transactions"],
    range_spec="@20241030:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2025-01-08 22:34:32 [warning  ] MISSING DATA                   chain=op date=2024-10-30 filename=bydate.py lineno=93 process=39190
2025-01-08 22:34:32 [info     ] prepared 1 input batches.      filename=bydate.py lineno=96 process=39190
2025-01-08 22:34:32 [info     ] input not ready for 1 batches. filename=bydate.py lineno=98 process=39190


In [12]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_context=duckdb_client,
    model="telepotr_bridging_transactions",
    data_reader=batch,
    limit_input_parquet_files=1
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2025-01-08 22:34:34 [info     ] Executing model function...    filename=testutils.py lineno=265 process=39190


KeyError: 'ingestion/logs_v1'